### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 1)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-02-01'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
0,BGRIM,40.25,0,"3,000","9,000",40.00,2022-01-27,2022-02-01,3,34.75,36.25,6,36.50,52.50,54.80,"120,750.00",I
10,CKP,5.50,0,"10,000","30,000",5.00,2022-01-31,2022-02-01,1,5.10,5.20,2,5.30,6.30,6.50,"55,000.00",I
1,IVL,50.50,12,"3,000","6,000",42.00,2022-01-26,2022-02-01,4,48.25,47.00,-5,48.00,50.00,57.78,"151,500.00",I
2,LPF,14.30,0,"10,000","10,000",14.00,2022-01-13,2022-02-01,13,14.00,12.70,-13,13.10,17.10,16.50,"143,000.00",I
11,MCS,14.70,3,"5,000","55,000",16.70,2022-01-31,2022-02-01,1,14.50,14.60,1,14.70,16.20,21.00,"73,500.00",I
3,NER,7.80,0,"9,000","27,000",7.45,2022-01-26,2022-02-01,4,7.05,7.40,7,7.45,8.65,11.35,"70,200.00",I
4,RJH,36.25,0,"1,500","4,500",33.00,2022-01-26,2022-02-01,4,32.25,32.75,2,33.00,39.00,36.50,"54,375.00",I
9,SYNEX,33.50,12,"3,000","9,000",30.50,2022-01-31,2022-02-01,1,29.75,30.00,1,31.00,37.00,31.87,"100,500.00",I
5,TOP,55.50,0,"2,000","4,000",50.50,2022-01-25,2022-02-01,5,50.75,52.75,8,53.00,66.00,65.65,"111,000.00",I
6,TSE,2.74,0,"40,000","40,000",2.60,2022-01-27,2022-02-01,3,2.50,2.56,3,2.60,3.70,3.43,"109,600.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['BGRIM', 40.25, 0, 3000, 9000, 40.0, '2022-01-27', '2022-02-01',
        3, 34.75, 36.25, 6, 36.5, 52.5, 54.8, 120750.0, 'I'],
       ['IVL', 50.5, 12, 3000, 6000, 42.0, '2022-01-26', '2022-02-01', 4,
        48.25, 47.0, -5, 48.0, 50.0, 57.78, 151500.0, 'I'],
       ['LPF', 14.3, 0, 10000, 10000, 14.0, '2022-01-13', '2022-02-01',
        13, 14.0, 12.7, -13, 13.1, 17.1, 16.5, 143000.0, 'I'],
       ['NER', 7.8, 0, 9000, 27000, 7.45, '2022-01-26', '2022-02-01', 4,
        7.05, 7.4, 7, 7.45, 8.65, 11.35, 70200.0, 'I'],
       ['RJH', 36.25, 0, 1500, 4500, 33.0, '2022-01-26', '2022-02-01', 4,
        32.25, 32.75, 2, 33.0, 39.0, 36.5, 54375.0, 'I'],
       ['TOP', 55.5, 0, 2000, 4000, 50.5, '2022-01-25', '2022-02-01', 5,
        50.75, 52.75, 8, 53.0, 66.0, 65.65, 111000.0, 'I'],
       ['TSE', 2.74, 0, 40000, 40000, 2.6, '2022-01-27', '2022-02-01', 3,
        2.5, 2.56, 3, 2.6, 3.7, 3.43, 109600.00000000001, 'I'],
       ['WHART', 12.8, 0, 10000, 10000, 12.5, '2022-01-17', '202

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([16, 14, 16,  8, 14, 11,  9, 13, 11, 14,  6,  1, 11])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,BGRIM,40.25,0,"3,000","9,000",40.00,2022-01-27,2022-02-01,3,34.75,36.25,6,36.50,52.50,54.80,"120,750.00",I,16
1,IVL,50.50,12,"3,000","6,000",42.00,2022-01-26,2022-02-01,4,48.25,47.00,-5,48.00,50.00,57.78,"151,500.00",I,14
2,LPF,14.30,0,"10,000","10,000",14.00,2022-01-13,2022-02-01,13,14.00,12.70,-13,13.10,17.10,16.50,"143,000.00",I,16
3,NER,7.80,0,"9,000","27,000",7.45,2022-01-26,2022-02-01,4,7.05,7.40,7,7.45,8.65,11.35,"70,200.00",I,8
4,RJH,36.25,0,"1,500","4,500",33.00,2022-01-26,2022-02-01,4,32.25,32.75,2,33.00,39.00,36.50,"54,375.00",I,14
5,TOP,55.50,0,"2,000","4,000",50.50,2022-01-25,2022-02-01,5,50.75,52.75,8,53.00,66.00,65.65,"111,000.00",I,11
6,TSE,2.74,0,"40,000","40,000",2.60,2022-01-27,2022-02-01,3,2.50,2.56,3,2.60,3.70,3.43,"109,600.00",I,9
7,WHART,12.80,0,"10,000","10,000",12.50,2022-01-17,2022-02-01,11,12.30,11.40,-9,11.70,14.08,14.90,"128,000.00",I,13
8,TU,21.80,12,"6,000","18,000",20.80,2022-01-28,2022-02-01,2,21.10,20.70,-4,21.00,23.00,25.56,"130,800.00",I,11
9,SYNEX,33.50,12,"3,000","9,000",30.50,2022-01-31,2022-02-01,1,29.75,30.00,1,31.00,37.00,31.87,"100,500.00",I,14


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
9,BCH,"15,000",21.30,"319,500.00",0.33,"4,950.00",1.55,3
28,BGRIM,"9,000",40.00,"360,000.00",0.45,"4,050.00",1.12,4
18,CKP,"30,000",5.00,"150,000.00",0.04,"1,050.00",0.70,3
5,DIF,"60,000",14.70,"882,000.00",1.04,"62,640.00",7.10,2
11,DOHOME,"12,000",24.30,"291,600.00",0.10,"1,200.00",0.41,3
25,EPG,"24,000",10.90,"261,600.00",0.23,"5,520.00",2.11,3
7,GLOBAL,"15,000",19.40,"291,000.00",nan,nan,nan,3
16,HREIT,"30,000",8.95,"268,500.00",0.69,"20,796.00",7.75,2
26,IMH,"9,000",17.60,"158,400.00",nan,nan,nan,3
33,IP,"7,500",20.20,"151,500.00",0.18,"1,387.50",0.92,3


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,BGRIM,16,40.25,0,"3,000","9,000",40.00,34.75,36.25,6,36.50,52.50,54.80,I,4,"120,750.00"
1,IVL,14,50.50,12,"3,000","6,000",42.00,48.25,47.00,-5,48.00,50.00,57.78,I,3,"151,500.00"
2,LPF,16,14.30,0,"10,000","10,000",14.00,14.00,12.70,-13,13.10,17.10,16.50,I,4,"143,000.00"
3,NER,8,7.80,0,"9,000","27,000",7.45,7.05,7.40,7,7.45,8.65,11.35,I,3,"70,200.00"
4,RJH,14,36.25,0,"1,500","4,500",33.00,32.25,32.75,2,33.00,39.00,36.50,I,4,"54,375.00"
5,TOP,11,55.50,0,"2,000","4,000",50.50,50.75,52.75,8,53.00,66.00,65.65,I,3,"111,000.00"
6,TSE,9,2.74,0,"40,000","40,000",2.60,2.50,2.56,3,2.60,3.70,3.43,I,3,"109,600.00"
7,WHART,13,12.80,0,"10,000","10,000",12.50,12.30,11.40,-9,11.70,14.08,14.90,I,4,"128,000.00"
8,TU,11,21.80,12,"6,000","18,000",20.80,21.10,20.70,-4,21.00,23.00,25.56,I,3,"130,800.00"
9,SYNEX,14,33.50,12,"3,000","9,000",30.50,29.75,30.00,1,31.00,37.00,31.87,I,3,"100,500.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
MCS,1,14.70,3,5000,55000,16.70,14.50,14.60,1,14.70,16.20,21.00,I,2,73500.0
CKP,6,5.50,0,10000,30000,5.00,5.10,5.20,2,5.30,6.30,6.50,I,3,55000.0
NER,8,7.80,0,9000,27000,7.45,7.05,7.40,7,7.45,8.65,11.35,I,3,70200.0
TSE,9,2.74,0,40000,40000,2.60,2.50,2.56,3,2.60,3.70,3.43,I,3,109600.0
TOP,11,55.50,0,2000,4000,50.50,50.75,52.75,8,53.00,66.00,65.65,I,3,111000.0
TU,11,21.80,12,6000,18000,20.80,21.10,20.70,-4,21.00,23.00,25.56,I,3,130800.0
TSTH,11,1.52,0,75000,75000,1.50,1.42,1.41,-1,1.43,2.44,1.50,I,4,114000.0
WHART,13,12.80,0,10000,10000,12.50,12.30,11.40,-9,11.70,14.08,14.90,I,4,128000.0
IVL,14,50.50,12,3000,6000,42.00,48.25,47.00,-5,48.00,50.00,57.78,I,3,151500.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(one_file)